# Исследование объявлений о продаже квартир
## Цель исследования:
        Провести иследование данных о квартирах в Иркутске предоставленных сайтом Циан.
## Задачи исследования:
        Доказать, что цена за квадратный метр зависит от района продажи квартиры 
## Декомпозиция работы:
        1. Подготовка данных 
        2. Предобработка данных
        3. Анализ данных
        4. Визуализация данных
        5. Вывод о задачах исследования

### 1. Подготовка данных 

In [8]:
#Импортируем библиотеки
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import requests
import io
# импорт библиотек для отоброжения графика на гитхаб
# import plotly.io as pio
# pio.renderers.default='notebook'

#показать всю информацию в колонках 
pd.set_option('display.max_colwidth', None)

#Загружаем данные 
# data = pd.read_csv('C:/Users/admin/all/flat_3.csv', sep=';')
url = "https://github.com/DanVik24/Project/blob/main/flat_3.csv"
s = requests.get(url).content
data = pd.read_csv(s)

display(data.head())


TypeError: Expected file path name or file-like object, got <class 'bytes'> type

In [ ]:
# переименование столбцов для удобного использования 
data = data.rename(columns={'Количество комнат': 'rooms', 
                               'Тип': 'type', 
                               'район': 'district', 
                               'Адрес': 'address', 
                               'Площадь, м2': 'area',
                               'Этаж': 'floor', 
                               'Цена': 'price', 
                               'Ремонт': 'repair', 
                               'Ссылка на объявление': 'link'})
# смотрим основную информацию о датафрейме
display(data.info())

In [ ]:
# просмотр уникальных данных в столбцах:
display(data['rooms'].unique())
display(data['type'].unique())
display(data['district'].unique())
display(data['repair'].unique())

In [ ]:
# изменим столбец с данными о колве квартир - уберем лишнюю информацию
data['rooms'] = data['rooms'].str.replace(',', ' ').str.replace('\D', '', regex=True) # удаляет данные в ячейке после ","
data.sample(5)

### 2. Предобработка данных
Перед работой с данными необходимо произвести следующие изменения и анализ данных:
##### 1. Создадим дополнительные столбцы 
Добавим столбцы "Этажей в доме" и столбец "Цена за квадратный метр":
1. Разделим столбец "floor": Возьмем из столбца "floor" информацию за знаком "/" и добавим столбец "floor_all" с данными о количестве этажей в доме
2. Добавим данные о цене за квадратный метр в новый столбец "price_square"


##### 2. Изменим типы объектов в следующих столбцах:
*rooms* - изменить на тип объекта int\
*floor* - изменить на int 

##### 3. Работа с пропусками и дубликатами в данных. 
1. Определим количество пропущенных значениях в столбцах и если есть пропуски, то для дальнейшей работы необходимо их убрать, заменив их на подходящие по логике данные. 
2. Проверим данные на дубликаты по столбцам "Адрес" и "ID" - там не должны быть дубли, если они будут то строки необходимо удалить

#### Разделим столбец "floor" и добавим столбец "price_square" с данными о цене за квадратный метр 

In [ ]:
# добавим столбец 'floor_all'
data['floor_all'] = data['floor'].str.split('/').str.get(1)
data['floor'] = data['floor'].str.split('/').str.get(0)
data['price_square'] = (data['price'] / data['area'])/1000 # создадим столбец "price_square" с данными о цене за кв.м. и 
data['price'] = data['price']/1000000  # поделим результат на 100 для лучшего восприятия цены

In [ ]:
data['price'] = data['price'].round(1) #  округлим числа до десятых
data['price_square'] = data['price_square'].round(1)
data.sample(5) 

Теперь данные в столбце "price" отображаются в млн., а данные в столбце "price_square" в тыс./за кв.м.

#### 2. Изменим типы объектов 

In [ ]:
# изменим тип данных для дальнейшего исследовани
data['rooms'] = data['rooms'].fillna(0).astype('int')
data['floor'] = data['floor'].fillna(0).astype('int')

In [ ]:
display(data.sample(5))  # проверим изменненые данные
display(data.info())

#### 3. Определим количество пропущенных значениях и исследуем на дубликаты в столбцах

In [ ]:
# посчитаем пропуски
print('Количество дубликатов:', data.duplicated().sum())
print('Количие пропусков:')
data.isnull().sum() 

Дубликатов данных нет. 
Пропуски в столбце с ремонтом и кол-вом комнат из-за отсутсивия информации - для работы с эти столбцом необходимо заменить пропущенные значения на 0 - как "нет информации о ремонте/комнатах":

In [ ]:
# замена пропусков
data['repair'] = data['repair'].fillna(0)

### 3. Анализ данных
#### Исследуем количество объектов по категориям: район продаж, количество комнат, площадь, цена за квадратный метр. Построим визуализации. 

In [ ]:
# районы
display(data['district'].value_counts())
data['district'].hist();

Больше всего объявлений о продаже в Октябрьском районе

In [ ]:
data['type'].unique() # посмотрим данные о типе квартир

Изменим столбец с данными о типе квартиры: заменим данные на "second" - вторичное жилье и "new" - новостройка.

In [ ]:
data['type']
data = data.replace({'type':{'Продажа квартиры в новостройке':"new", 'Продажа квартиры': "second"}})
data.head()

In [ ]:
data['rooms'].unique()

In [ ]:
data[data['rooms'] == 0] 

По полученным данным большенство квартир с кол-вом комнат = 0 это новостройки без ремонта. Остальные 5 квартир имеют площадь менее 31 квадрата - значит это студии. Оставим как есть.

In [ ]:
display(data['rooms'].value_counts()) #посчитаем количество квартир в зависимости от количества комнат
data['rooms'].hist(figsize=(4,4)) #построим диаграмму 

In [ ]:
#сгруппируем данные по району
data_district = data.pivot_table(index = 'district', values = 'ID', columns='rooms', aggfunc = 'count').reset_index().fillna(0)
display(data_district) 
display(data_district.info())

### 4. Визуализация данных
#### Визуализация данных по районам

In [ ]:
# строим гистограммы 
fig = px.histogram(data_district, # загружаем данные 
                   x=[0, 1, 2, 3, 4, 5], # указываем столбец с данными для оси X 
                   y= 'district',
                   title='Распределение по районам') # указываем заголовок 
fig.update_layout(yaxis_title ='Районы') # подпись для оси X 
fig.update_layout(xaxis_title ='Количество комнат') # подпись для оси Y 
fig.show("png"); # выводим график

В полученном результате видим преобладание объявлений о продаже однокомнатных квартир в любом из районов

In [ ]:
# можем посмотреть колличество каких квартир продается, 
# новостроек или вторичного жилья, по районам
data_price4 = data.pivot_table(index = 'district', values = 'ID', columns='type', aggfunc = 'count').reset_index()
data_price4['%_new'] = 100*(data_price4['new'] / data_price4['new'].sum()).round(3)
data_price4['%_second'] = 100*(data_price4['second'] / data_price4['second'].sum()).round(3)

data_price4

Новостроек много в Правобережном районе - 28%. А объявлений вторичного жилья больше в Ленинском районе - 32,4%.

In [ ]:
# можем посмотреть колличество скольки комнатных квартир, 
# новостроек или вторичного жилья, в каких районах продается больше чем в остальных
data_price2 = data.pivot_table(index = ['district', 'rooms'], values = 'ID', columns='type', aggfunc = 'count')
data_price2

#### Посмотрим заполнение данных в столбце с данными - цена за кв.метр

In [ ]:
# цена за квартиру
data['price'].describe().round(3)

In [ ]:
# цена за квадратный метр
display(data['price_square'].describe().round(1))
data['price_square'].hist(bins=100);

   Средняя цена за квадратный метр - 67.3 т.р./кв. А максиматльная сумма за квадрат - 346.2 т.р./кв Посмотрим эти объявления:

In [ ]:
data[data['price_square'] == 67.3] 

In [ ]:
data[data['price_square'] == 346.2] 

Для просмотра квартир по районам с ценой за квадратный метр создадим категории в датафрейме data - столбец cat_pricesquare:

0–119 — 'менее 119 т.р за кв.м';\
120–140 — 'от 120 до 140 т.р за кв.м';\
141–152 — 'от 141 до 152 т.р за кв.м';\
152 и выше — 'свыше 152 т.р за кв.м'.

In [ ]:
# создадим функцию для категорий по цене за кв.м.
def cat_pricesquare(income):
    try:
        if 0 <= income <= 119:
            return 'менее 119 т.р за кв.м'
        elif 120 <= income <= 140:
            return 'от 120 до 140 т.р за кв.м'
        elif 141 <= income <= 152:
            return 'от 141 до 152 т.р за кв.м'
        elif income >= 153:
            return 'свыше 152 т.р за кв.м'
    except:
        pass

In [ ]:
# создадим столбец и применю функцию
data['cat_pricesquare'] = data['price_square'].apply(cat_pricesquare)
display(data.sample(5)) 

In [ ]:
# создадим сводную таблицу по кол-ву обявлений
data_price5 = data.pivot_table(index = 'district', values = 'ID', columns='cat_pricesquare', aggfunc = 'count').reset_index()
data_price5

Дорогие квартиры, с ценой более 152тр. за кв.м. в Октябрьском районе, а в Ленинском - самые не дорогие с ценой до 119т.р. за кв.м.

In [ ]:
# строим гистограммы 
fig = px.histogram(data_price5, # загружаем данные 
                   x=["менее 119 т.р за кв.м", "от 120 до 140 т.р за кв.м", "от 141 до 152 т.р за кв.м", "свыше 152 т.р за кв.м"], # указываем столбец с данными для оси X 
                   y= 'district',
                   title='Распределение по цене за квадрат') # указываем заголовок 
fig.update_layout(yaxis_title ='Районы') # подпись для оси X 
fig.update_layout(xaxis_title ='Цена за квадратный метр') # подпись для оси Y 
fig.show("png"); # выводим график

## Вывод:
* Больше всего объявлений о продаже в Октябрьском районе
* Преобладание объявлений о продаже однокомнатных квартир в любом из районов
* Новостроек много в Правобережном районе - 28%. А объявлений вторичного жилья больше в Ленинском районе - 32,4%.
* Дорогие квартиры, с ценой более 152тр. за кв.м. в Октябрьском районе, а в Ленинском - самые не дорогие с ценой до 119т.р. за кв.м.

#### По последней визуализации видно, что количество предложений недорогих квартир в Ленинском и Свердловском районе. А преобладание более дорогих квартир от 152тр за кв.м. - в Октябрьском районе. Из чего следует вывод, что цена за квадратный метр зависит от района продажи